In [30]:
import numpy as np
import matplotlib.pyplot as plt
import ase
from ase.visualize import view

data = np.load("../EF/data-full.npz", allow_pickle=True)
keys = list(data.keys())
keys
R = data["R"]
P = data["P"]
Z = data["Z"]

In [41]:
try:
    from scipy.spatial import cKDTree
    _HAS_SCIPY = True
except Exception:
    _HAS_SCIPY = False


def oriented_volume(p1, p2, p3, p4):
    """
    Oriented (signed) volume of tetrahedron (p1, p2, p3, p4).
    Returns OV = (1/6) * (p1-p4) · ((p2-p4) × (p3-p4))
    """
    p1 = np.asarray(p1, dtype=float)
    p2 = np.asarray(p2, dtype=float)
    p3 = np.asarray(p3, dtype=float)
    p4 = np.asarray(p4, dtype=float)

    a = p1 - p4
    b = p2 - p4
    c = p3 - p4

    return (1.0 / 6.0) * np.dot(a, np.cross(b, c))


def oriented_volume_det(p1, p2, p3, p4):
    """
    Same OV computed via the 4x4 determinant:
    OV = (1/6) * det([[1,1,1,1],
                     [x1,x2,x3,x4],
                     [y1,y2,y3,y4],
                     [z1,z2,z3,z4]])
    """
    p1 = np.asarray(p1, dtype=float)
    p2 = np.asarray(p2, dtype=float)
    p3 = np.asarray(p3, dtype=float)
    p4 = np.asarray(p4, dtype=float)

    M = np.array([
        [1,     1,     1,     1],
        [p1[0], p2[0], p3[0], p4[0]],
        [p1[1], p2[1], p3[1], p4[1]],
        [p1[2], p2[2], p3[2], p4[2]],
    ], dtype=float)

    return (1.0 / 6.0) * np.linalg.det(M)


def four_nearest_neighbors(positions, k=4):
    """
    Returns (neighbors, distances)
      neighbors: (N, k) int indices of k nearest atoms for each atom (excluding itself)
      distances: (N, k) float distances corresponding to neighbors
    """
    pos = np.asarray(positions, dtype=float)
    n = pos.shape[0]
    if pos.shape[1] != 3:
        raise ValueError("positions must be shape (N, 3)")

    if _HAS_SCIPY:
        tree = cKDTree(pos)
        # query k+1 because the closest is the atom itself at distance 0
        dists, idxs = tree.query(pos, k=k + 1)
        dists = dists[:, 1:]
        idxs = idxs[:, 1:]
        return idxs, dists

    # Fallback O(N^2): OK for small N
    diffs = pos[:, None, :] - pos[None, :, :]
    dmat = np.linalg.norm(diffs, axis=2)
    np.fill_diagonal(dmat, np.inf)
    idxs = np.argsort(dmat, axis=1)[:, :k]
    dists = np.take_along_axis(dmat, idxs, axis=1)
    return idxs, dists

In [32]:
R[0], Z[0]

(array([[[-0.856624, 0.129623, 1.934218],
         [-1.113226, 0.208861, 0.29253],
         [-2.395473, -0.129327, -0.244816],
         [-2.758218, 0.292541, -1.608939],
         [-3.369716, -0.937218, 0.535769],
         [0.00851, 0.750542, -0.561608],
         [0.074188, 2.289299, -0.495448],
         [1.530449, 2.646581, -0.349334],
         [2.25431, 1.399645, 0.131496],
         [1.32012, 0.278177, -0.140145],
         [1.720577, -1.093701, -0.088373],
         [0.86284, -2.237621, -0.873717],
         [3.046157, -1.479629, 0.505688],
         [-3.842935, 0.16159, -1.799584],
         [-2.564452, 1.387345, -1.717807],
         [-2.178615, -0.257288, -2.347192],
         [-2.88315, -1.865078, 0.871457],
         [-4.253185, -1.197732, -0.071191],
         [-3.687413, -0.336486, 1.41572],
         [-0.110561, 0.468629, -1.607865],
         [-0.39813, 2.77545, -1.388399],
         [-0.48387, 2.673344, 0.418778],
         [1.943412, 2.942095, -1.384198],
         [1.714216, 3.502091, 

In [37]:
index = 0
atoms = ase.Atoms( Z[index], R[index][0])

In [38]:
view(atoms, viewer="x3d")

In [35]:
handedness = np.tile(np.array([1, -1,]), len(R)//2)

In [36]:
out_data = {"R": R, "Z": Z, "handedness": handedness}
np.savez("handedness.npz", **out_data)

In [42]:
four_nearest_neighbors(R[0][0])

(array([[ 1,  2,  5, 18],
        [ 2,  5,  0, 19],
        [ 1,  3,  4, 18],
        [15, 13, 14,  2],
        [16, 17, 18,  2],
        [19,  9,  1,  6],
        [20, 21,  7,  5],
        [23, 22,  6,  8],
        [24, 25,  9,  7],
        [10,  5,  8, 19],
        [ 9, 12, 11, 27],
        [10,  9, 12, 27],
        [27, 26, 28, 10],
        [ 3, 14, 15,  2],
        [ 3, 13, 15,  2],
        [ 3, 13, 14,  2],
        [ 4, 17, 18,  2],
        [ 4, 16, 18,  2],
        [ 4, 17, 16,  2],
        [ 5,  9,  6,  1],
        [ 6, 21,  7,  5],
        [ 6, 20,  7,  5],
        [ 7, 23,  6,  8],
        [ 7, 22,  8,  6],
        [ 8, 25,  9,  7],
        [ 8, 24,  9,  7],
        [12, 27, 28, 10],
        [12, 26, 28, 10],
        [12, 27, 26, 10]]),
 array([[1.66350916, 2.680168  , 2.7135115 , 2.91538405],
        [1.43082816, 1.51038461, 1.66350916, 2.16432846],
        [1.43082816, 1.47322371, 1.48699369, 2.11409641],
        [1.08778171, 1.10910083, 1.11713621, 1.47322371],
        [1.1

In [47]:
R_ = R[index][0]
Z_ = Z[index]
neighbours, distances = four_nearest_neighbors(R_)

for z, ns in zip(Z_, neighbours):
    _ = R_[ns]
    print(z, oriented_volume_det(*_))

16 -0.15348372815114586
6 0.3569074532518456
7 0.8881474898788633
6 -0.8616158217092682
6 0.8714416408484467
6 -1.3752966107669127
6 -1.1690450446152185
6 -1.1665203033996052
6 -1.1331980804176531
7 0.8787347137812944
6 -0.3889655039297366
16 -0.10018200954874674
6 0.8889750687327254
1 -0.2265940998064958
1 0.21643364214764005
1 -0.23984477429768597
1 0.23243734446680256
1 -0.237004865555278
1 -0.22861339448205792
1 0.42415123040820224
1 -0.35102286470379596
1 0.32119071853776976
1 0.31495659929353187
1 0.3528879100705822
1 -0.341740552711906
1 0.3224028779881818
1 -0.23655521845327382
1 0.23408128364147351
1 0.23652367658435325


In [51]:
R_ = R[index+1][0]
Z_ = Z[index]
neighbours, distances = four_nearest_neighbors(R_)

for z, ns in zip(Z_, neighbours):
    _ = R_[ns]
    print(z, oriented_volume_det(*_))

16 0.17674344042208723
6 -0.3706639277797472
7 -0.8856681470252097
6 -0.8656042012772157
6 -0.8738198330784479
6 -1.3708615933994803
6 -1.1461982350528186
6 1.1298674788675926
6 1.1098496226250136
7 0.8908409848367527
6 0.6474504466973592
16 0.10194360721734158
6 0.90314866218864
1 0.22902661788819778
1 0.23216060984197265
1 -0.2293628722644714
1 -0.22738063715458962
1 -0.22662811430824037
1 0.2356690897676536
1 -0.42759431742441373
1 -0.3344999942127853
1 -0.32526764857614565
1 0.3201753684600069
1 -0.3408498414115759
1 -0.32008769105785134
1 0.3369138814072905
1 0.233982303605524
1 -0.2450854262515647
1 -0.2414248952722624
